* conda install pytorch==2.4.0 torchvision==0.19.0 torchaudio==2.4.0 -c pytorch
* !pip install datasets  
* !pip install python-dotenv
* !pip install openai
* !pip install crewai


In [1]:
# 100분 / 0.27달러 
import openai
from datasets import load_dataset
import random

# 시드 고정
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

import os 
from dotenv import load_dotenv
load_dotenv()
model_name = "gpt-4o-mini"
api_key = os.environ.get("OPENAI_API_KEY")
huggingface_token = os.environ.get("HUGGINGFACE_TOKEN")


from openai import OpenAI
client = OpenAI(
  api_key=api_key
)

corrot_prompt = """당신은 가상의 중고나라 게시물을 생성해야 합니다.

이름과 연락처(전화번호, 카카오톡, 거래 위치 등...)를 남기고 어떤 상품을 판매하고 있는지를 작성하세요.

- 전화번호는 반드시 5자리(010-87590-12345) 형식으로 작성하고, **로 감싸세요. (예시: *010-87590-12345*)
- 이메일은 실제 존재하지 않도록 `_sudo_`를 포함하여 작성하세요. (예시: *en_sudo_user@example.com*, *user_sudo@semi.com* 등 다양한 패턴)
- 카카오톡, 트위터, 텔레그램 등 다양한 아이디를 사용하되 사용되는 아이디에는 반드시 실제 존재하지 않도록 `_sudo_`를 포함하세요. (예시: *data_insight_sudo* 등 다양한 패턴)
- 주소를 생성할 때는 반드시 **로 감싸세요. (예시: *서울시 강남구 테헤란로 123, ABC 강의실*)
- 글머리 기호를 써도 되지만 남용해선 안 됩니다. 일반적인 블로그, 카페 말투를 쓰십시오.
- 개인정보나 연락처를 작성할 때만큼은 글머리 기호나 나열식 방식을 쓰지말고 평문으로 이어서 쓰십시오.

당신이 참고할 게시물입니다. 아래 게시물로부터 아이디어를 얻고 작성하세요.
---
{new_dataset}
---

작성:"""

event_prompt = """당신은 가상의 이벤트 홍보글을 생성해야 합니다.

이벤트명, 일시, 장소, 참가 방법, 주최자 연락처(이메일, 전화번호 등), 이벤트 상세 내용 등을 작성하세요.

- 전화번호는 반드시 5자리(010-87590-12345) 형식으로 작성하고, **로 감싸세요. (예시: *010-87590-12345*)
- 이메일은 실제 존재하지 않도록 `_sudo_`를 포함하여 작성하세요. (예시: *en_sudo_user@example.com*, *user_sudo@semi.com* 등 다양한 패턴)
- 카카오톡, 트위터, 텔레그램 등 다양한 아이디를 사용하되 사용되는 아이디에는 반드시 실제 존재하지 않도록 `_sudo_`를 포함하세요. (예시: *data_insight_sudo* 등 다양한 패턴)
- 주소를 생성할 때는 반드시 **로 감싸세요. (예시: *서울시 강남구 테헤란로 123, ABC 강의실*)
- 글머리 기호를 써도 되지만 남용해선 안 됩니다. 친근하고 설득력 있는 말투를 사용하십시오.
- 연락처나 개인정보를 작성할 때는 글머리 기호나 나열식 방식을 쓰지말고 평문으로 이어서 쓰십시오.

당신이 참고할 이벤트 홍보글입니다. 아래 게시물로부터 아이디어를 얻고 작성하세요.
---
{new_dataset}
---

작성:"""

lecture_prompt = """당신은 가상의 강의 또는 세미나 홍보글을 생성해야 합니다.

강의명, 강사명, 일시, 장소, 수강료, 신청 방법, 주최자 연락처(이메일, 전화번호 등), 강의 상세 내용 등을 작성하세요.

- 전화번호는 반드시 5자리(010-87590-12345) 형식으로 작성하고, **로 감싸세요. (예시: *010-87590-12345*)
- 이메일은 실제 존재하지 않도록 `_sudo_`를 포함하여 작성하세요. (예시: *en_sudo_user@example.com*, *user_sudo@semi.com* 등 다양한 패턴)
- 카카오톡, 트위터, 텔레그램 등 다양한 아이디를 사용하되 사용되는 아이디에는 반드시 실제 존재하지 않도록 `_sudo_`를 포함하세요. (예시: *data_insight_sudo* 등 다양한 패턴)
- 주소를 생성할 때는 반드시 **로 감싸세요. (예시: *서울시 강남구 테헤란로 123, ABC 강의실*)
- 글머리 기호를 써도 되지만 남용해선 안 됩니다. 전문적이고 신뢰감 있는 말투를 사용하십시오.
- 연락처나 개인정보를 작성할 때는 글머리 기호나 나열식 방식을 쓰지말고 평문으로 이어서 쓰십시오.

당신이 참고할 강의/세미나 홍보글입니다. 아래 게시물로부터 아이디어를 얻고 작성하세요.
---
{new_dataset}
---

작성:"""

product_prompt = """당신은 가상의 제품 리뷰 또는 홍보글을 생성해야 합니다.

제품명, 가격, 구매처, 제품 특징, 사용 후기, 연락처(구매 문의처 등) 등을 작성하세요.

- 전화번호는 반드시 5자리(010-87590-12345) 형식으로 작성하고, **로 감싸세요. (예시: *010-87590-12345*)
- 이메일은 실제 존재하지 않도록 `_sudo_`를 포함하여 작성하세요. (예시: *en_sudo_user@example.com*, *user_sudo@semi.com* 등 다양한 패턴)
- 카카오톡, 트위터, 텔레그램 등 다양한 아이디를 사용하되 사용되는 아이디에는 반드시 실제 존재하지 않도록 `_sudo_`를 포함하세요. (예시: *data_insight_sudo* 등 다양한 패턴)
- 주소를 생성할 때는 반드시 **로 감싸세요. (예시: *서울시 강남구 테헤란로 123, ABC 강의실*)
- 글머리 기호를 써도 되지만 남용해선 안 됩니다. 솔직하고 친근한 말투를 사용하십시오.
- 연락처나 개인정보를 작성할 때는 글머리 기호나 나열식 방식을 쓰지말고 평문으로 이어서 쓰십시오.

당신이 참고할 제품 리뷰/홍보글입니다. 아래 게시물로부터 아이디어를 얻고 작성하세요.
---
{new_dataset}
---

작성:"""

meeting_prompt = """당신은 가상의 모임 또는 동호회 홍보글을 생성해야 합니다.

모임명, 활동 내용, 모임 일시, 장소, 가입 방법, 연락처(이메일, 전화번호 등) 등을 작성하세요.

- 전화번호는 반드시 5자리(010-87590-12345) 형식으로 작성하고, **로 감싸세요. (예시: *010-87590-12345*)
- 이메일은 실제 존재하지 않도록 `_sudo_`를 포함하여 작성하세요. (예시: *en_sudo_user@example.com*, *user_sudo@semi.com* 등 다양한 패턴)
- 카카오톡, 트위터, 텔레그램 등 다양한 아이디를 사용하되 사용되는 아이디에는 반드시 실제 존재하지 않도록 `_sudo_`를 포함하세요. (예시: *data_insight_sudo* 등 다양한 패턴)
- 주소를 생성할 때는 반드시 **로 감싸세요. (예시: *서울시 강남구 테헤란로 123, ABC 강의실*)
- 글머리 기호를 써도 되지만 남용해선 안 됩니다. 친근하고 열정적인 말투를 사용하십시오.
- 연락처나 개인정보를 작성할 때는 글머리 기호나 나열식 방식을 쓰지말고 평문으로 이어서 쓰십시오.

당신이 참고할 모임/동호회 홍보글입니다. 아래 게시물로부터 아이디어를 얻고 작성하세요.
---
{new_dataset}
---

작성:"""

anonymization_prompt = """다음 텍스트의 개인정보를 아래 규칙에 따라 비식별화해주세요:

1. 이메일 주소는 @앞부분을 X로 마스킹하고 도메인은 유지해주세요.
   예시: example@gmail.com -> *[EMAIL]*

2. 전화번호는 모두 숫자 부분을 X로 변경해주세요.
   예시: 010-12334-56783 -> *[PHONE_NUMBER]*

3. 카카오톡 ID, 사용자 ID 등의 ID 전체를 X로 마스킹해주세요.
   예시: user_sudo123 -> *[ID]*
   예시: @SmartSec_sudo -> *[ID]*

4. 주소는 아래 예시와 같이 제거해주세요.
   예시: 거래는 *서울 강남역* -> *[ADDR]*
   예시: *서울시 강남구 청담동 123-45, 청담 커뮤니티 센터 2층* -> *[ADDR]*
   예시: *서울시 강남구 테헤란로 123, ABC 강의실* -> *[ADDR]*
   예시: *서울시 청년센터 1층 대강당* -> *[ADDR]*

5. 사람 이름도 마스킹해주세요. 
   예시: 홍길동 -> *[NAME]*

6. 비식별화 예시 
### 원본 데이터 
# **이벤트명: 우리 동네 청년 창업 페스티벌**

안녕하세요! 여러분을 위한 특별한 기회를 소개합니다. 창업에 관심 있는 청년들을 위한 '우리 동네 청년 창업 페스티벌'이 열립니다! 

**일시:** 2023년 11월 15일 (수) 오전 10시부터 오후 5시까지  
**장소:** *서울시 청년센터 1층 대강당*

이번 페스티벌에서는 창업 전문가의 강연, 실전 창업 멘토링, 네트워킹 기회를 제공합니다. 다양한 부스와 체험 프로그램을 통해 창업에 대한 실질적인 정보를 얻고, 새로운 아이디어를 얻어가실 수 있습니다. 참여하시는 모든 분들께 소정의 기념품도 제공하니 많은 관심 부탁드립니다!

참가 방법은 간단합니다. 아래의 이메일로 참여 신청을 해주시면 됩니다. 신청은 선착순으로 마감되니 서둘러주세요. 이메일은 *youth_festival_sudo@example.com*입니다. 

혹시 궁금한 점이 있으신가요? 주최자에게 연락 주시면 친절히 안내해 드리겠습니다. 전화번호는 *02-12345-67890*이며, 카카오톡 아이디는 *youth_event_sudo*입니다. 텔레그램으로는 *youth_festival_sudo*를 통해 소통하실 수 있습니다. 트위터에서는 *youth_festival_sudo*를 검색해 주시면 됩니다.

여러분의 참여를 기다립니다! 창업의 꿈을 실현하는 첫걸음을 함께해요!

### 비식별화 데이터 
**이벤트명: 우리 동네 청년 창업 페스티벌**

안녕하세요! 여러분을 위한 특별한 기회를 소개합니다. 창업에 관심 있는 청년들을 위한 '우리 동네 청년 창업 페스티벌'이 열립니다! 

**일시:** 2023년 11월 15일 (수) 오전 10시부터 오후 5시까지  
**장소:** *[ADDR]*

이번 페스티벌에서는 창업 전문가의 강연, 실전 창업 멘토링, 네트워킹 기회를 제공합니다. 다양한 부스와 체험 프로그램을 통해 창업에 대한 실질적인 정보를 얻고, 새로운 아이디어를 얻어가실 수 있습니다. 참여하시는 모든 분들께 소정의 기념품도 제공하니 많은 관심 부탁드립니다!

참가 방법은 간단합니다. 아래의 이메일로 참여 신청을 해주시면 됩니다. 신청은 선착순으로 마감되니 서둘러주세요. 이메일은 *[EMAIL]*입니다. 

혹시 궁금한 점이 있으신가요? 주최자에게 연락 주시면 친절히 안내해 드리겠습니다. 전화번호는 *[PHONE_NUMBER]*이며, 카카오톡 아이디는 *[ID]*입니다. 텔레그램으로는 *[ID]*를 통해 소통하실 수 있습니다. 트위터에서는 *[ID]*를 검색해 주시면 됩니다.

여러분의 참여를 기다립니다! 창업의 꿈을 실현하는 첫걸음을 함께해요!
---
시작:
{new_dataset}

위 규칙에 따라 개인정보가 비식별화된 텍스트를 생성해주세요."""


def format_prompt(base_prompt: str, reference_text: str) -> str:
    """
    base_prompt에서 {new_dataset} 부분에 reference_text를 치환하여 반환합니다.
    """
    return base_prompt.format(new_dataset=reference_text)

def generate_text_with_prompt(prompt_text: str) -> str:
    """
    ChatCompletion API 호출 및 응답 텍스트 반환.
    """
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {
                    "role": "system", 
                    "content": "You are ChatGPT, a large language model trained by OpenAI."
                },
                {
                    "role": "user", 
                    "content": prompt_text
                }
            ],
            temperature=0.7,
            max_tokens=512
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"OpenAI API Error: {e}")
        return ""

# -------------------------
# (3) 실행부 (메인)
# -------------------------

# (1) 데이터셋 로드 (train 스플릿 예시)
ds = load_dataset("daekeun-ml/naver-news-summarization-ko", split="train")

# 예시로 'summary' 컬럼을 사용
sample_column = "summary"

# (4-2) 5가지 프롬프트를 리스트로 관리
prompts = [
    corrot_prompt,
    event_prompt,
    lecture_prompt,
    product_prompt,
    meeting_prompt
]

generated_data = []
sample_size = 500  # 예시로 5개만 생성

for i in range(sample_size):
    # 시드 고정되어 있으므로 매번 같은 결과 샘플링
    random_idx = random.randint(0, len(ds) - 1)
    reference_item = ds[random_idx][sample_column]
    
    # 5가지 프롬프트 중 하나를 랜덤 선택
    selected_prompt = random.choice(prompts)

    # 실제 프롬프트 구성
    prompt_for_openai = format_prompt(selected_prompt, reference_item)
    
    # OpenAI API 호출 (생성 텍스트)
    generated_text = generate_text_with_prompt(prompt_for_openai)
    
    # (4-3) 우선 generated_data에 저장
    generated_data.append({
        "reference_text": reference_item,
        "generated_text": generated_text
    })

# (4-4) 생성된 텍스트를 비식별화
for idx, item in enumerate(generated_data):
    # 비식별화용 프롬프트 생성
    anonym_prompt = format_prompt(anonymization_prompt, item["generated_text"])
    
    try:
        response = client.chat.completions.create(
            model=model_name,
            messages=[
                {
                    "role": "system", 
                    "content": "당신은 비식별화 전문가입니다. 주어진 원문을 규칙에 맞춰 비식별화하세요."
                },
                {
                    "role": "user", 
                    "content": anonym_prompt
                }
            ],
            temperature=0.7,
            max_tokens=512
        )
        # 비식별화된 텍스트 저장
        anonymized_text = response.choices[0].message.content
    except Exception as e:
        print(f"OpenAI API Error (anonymization): {e}")
        anonymized_text = ""

    # 결과를 generated_data에 추가
    item["비식별화된 데이터"] = anonymized_text

# (4-5) 최종 결과 출력
for idx, item in enumerate(generated_data):
    print(f"=== 샘플 {idx+1} ===")
    print("[원문(참조):]")
    print(item["reference_text"])
    print("\n[생성된 텍스트:]")
    print(item["generated_text"])
    print("\n[비식별화된 데이터:]")
    print(item["비식별화된 데이터"])
    print("-" * 40)
    if idx > 5 : 
        break 

from datasets import Dataset 
import pandas as pd 

created_ds = Dataset.from_list(generated_data)
created_ds = pd.DataFrame(generated_data)

from datetime import datetime 
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
created_ds.to_csv(f"./비식별화데이터_{timestamp}.csv", index=False)
print("데이터를 모두 생성하였습니다.")

/root/anaconda3/envs/chapter10/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


=== 샘플 1 ===
[원문(참조):]
KT는 양자역학 원리를 적용해 빛의 가장 작은 단위인 광자에 정보를 담아 암호화해 전송하는 차세대 보안 통신 기술인 양자암호 전용회선 서비스를 이달 1일 출시하고 공공분야와 단체 기업 등을 대상으로 한 B2B 영업을 본격적으로 시작한다고 10일 밝혔다.

[생성된 텍스트:]
안녕하세요! 중고나라에 올립니다.

저는 사용하지 않는 노트북을 판매하고자 합니다. 제품은 Dell XPS 13 모델로, 사양은 i7 프로세서, 16GB RAM, 512GB SSD입니다. 상태는 아주 양호하며, 구매 후 1년 정도 사용했습니다. 기본 구성품 모두 포함되어 있습니다. 가격은 70만원이며, 가격은 조정 가능합니다. 

관심 있으신 분들은 연락 주시면 됩니다. 전화번호는 *010-87590-12345*이고, 카카오톡 ID는 *dell_laptop_sudo*입니다. 거래는 *서울시 용산구 이태원로 45, 카페 A*에서 진행하길 원합니다. 이메일 문의는 *user_sudo@example.com*으로 주시면 됩니다.

감사합니다!

[비식별화된 데이터:]
안녕하세요! 중고나라에 올립니다.

저는 사용하지 않는 노트북을 판매하고자 합니다. 제품은 Dell XPS 13 모델로, 사양은 i7 프로세서, 16GB RAM, 512GB SSD입니다. 상태는 아주 양호하며, 구매 후 1년 정도 사용했습니다. 기본 구성품 모두 포함되어 있습니다. 가격은 70만원이며, 가격은 조정 가능합니다.

관심 있으신 분들은 연락 주시면 됩니다. 전화번호는 *[PHONE_NUMBER]*이고, 카카오톡 ID는 *[ID]*입니다. 거래는 *[ADDR]*에서 진행하길 원합니다. 이메일 문의는 *[EMAIL]*으로 주시면 됩니다.

감사합니다!
----------------------------------------
=== 샘플 2 ===
[원문(참조):]
4시 공시에 따르면 엔시트론 101400 은 투자 목적으로 50억 규모의 HLB글로벌 무기명식 이권부 무보증 사모 비분리형

In [44]:
df = pd.read_csv("./비식별화데이터.csv")
df

,reference_text,generated_text,비식별화된 데이터
0,KT는 양자역학 원리를 적용해 빛의 가장 작은 단위인 광자에 정보를 담아 암호화해 ...,안녕하세요! 중고나라에 판매글을 올립니다.\n\n저는 현재 사용하지 않는 삼성 Ga...,안녕하세요! 중고나라에 판매글을 올립니다.\n\n저는 현재 사용하지 않는 삼성 Ga...
1,4시 공시에 따르면 엔시트론 101400 은 투자 목적으로 50억 규모의 HLB글로...,**강의명:** 데이터 분석의 기초와 응용\n\n**강사명:** 김지훈 박사\n\n...,**강의명:** 데이터 분석의 기초와 응용\n\n**강사명:** XXX 박사\n\n...
2,국민 국민의힘 물가민생안정 특별위원회는 5일 국회에서 정부 전문가들과 주거 비용 안...,**이벤트명**: 2023 청년 창업 박람회\n\n**일시**: 2023년 11월 ...,**이벤트명**: 2023 청년 창업 박람회\n\n**일시**: 2023년 11월 ...
3,국내 독자기술로 개발한 한국형발사체 누리호의 원천 기술을 이어 받을 민간 업체 선정...,안녕하세요! 중고나라에 올라온 중고 물품 판매 글입니다. \n\n저는 최근에 새로 ...,안녕하세요! 중고나라에 올라온 중고 물품 판매 글입니다.\n\n저는 최근에 새로 구...
4,한 성공을 거뒀지만 현재 성공에 안주하지 말고 국가를 대표한다는 책임감하에 누리호 ...,"**🎉 함께하는 즐거움, '우리 마을 독서 모임'에 초대합니다! 📚**\n\n안녕하...","**🎉 함께하는 즐거움, '우리 마을 독서 모임'에 초대합니다! 📚**\n\n안녕하..."
...,...,...,...
495,유럽시간으로 5일 유럽의회가 전체회의를 열고 디지털서비스법과 디지털시장법을 최종 승...,**강의명:** 디지털 플랫폼 규제의 이해와 대응 전략 \n**강사명:** 김영호...,**강의명:** 디지털 플랫폼 규제의 이해와 대응 전략 \n**강사명:** XXX...
496,현대차그룹은 지난 5월 전국 대학생을 대상으로 선발한 환경 봉사단 100명이 참석한...,"**📦 제품명: 에코워터 필터 시스템** \n**💰 가격: 149,000원** ...","**📦 제품명: 에코워터 필터 시스템** \n**💰 가격: 149,000원** ..."
497,5일 현지시간 ‘수학계의 노벨상’으로 불리는 필즈상을 수상한 허준이 39 미국 프린...,"**제품명**: 스마트 홈 자동화 시스템 ""스마트홈 360"" \n**가격**: 2...","**제품명**: 스마트 홈 자동화 시스템 ""스마트홈 360"" \n**가격**: 2..."
498,"1일피너스와, LG에너지솔루션, 카카오뱅크 등 코스피 대형주들이 장중 일제히 52...",🌟 **2023 서울 글로벌 스타트업 엑스포** 🌟\n\n📅 **일시:** 2023...,🌟 **2023 서울 글로벌 스타트업 엑스포** 🌟\n\n📅 **일시:** 2023...


In [119]:
print(df.iloc[20]["generated_text"])

**강의명:** 효과적인 데이터 분석을 위한 Python 입문

**강사명:** 김지훈 데이터 사이언스 전문가

**일시:** 2023년 11월 15일 (수) 14:00 - 17:00

**장소:** 서울시 강남구 테헤란로 123, ABC 강의실

**수강료:** 50,000원

**신청 방법:** 아래의 이메일로 신청서를 제출해 주시기 바랍니다. 선착순 마감입니다. en_sudo_user@example.com

**주최자 연락처:** 전화번호 010-87590-12345, 카카오톡 ID: test_sudo_kakao, 트위터: @data_sudo_twitter, 텔레그램: @data_sudo_telegram

**강의 상세 내용:** 본 강의는 데이터 분석의 기본 개념과 Python 프로그래밍 언어를 활용하여 데이터 분석을 수행하는 방법을 배우는 프로그램입니다. 강의는 데이터 전처리, 시각화 및 분석 기법을 포함하며, 실습을 통해 이론을 체득할 수 있도록 구성되어 있습니다. 초보자도 쉽게 따라할 수 있도록 단계별로 설명하며, 실제 사례를 통해 학습의 이해도를 높입니다. 

데이터 분석에 관심 있는 분들, 취업 준비생 및 직장인 분들의 많은 참여를 바랍니다.


In [120]:
print(df.iloc[20]["비식별화된 데이터"])

다음은 주어진 원문을 비식별화한 텍스트입니다:

**강의명:** 효과적인 데이터 분석을 위한 Python 입문

**강사명:** XXX 데이터 사이언스 전문가

**일시:** 2023년 11월 15일 (수) 14:00 - 17:00

**장소:** 서울시 강남구 XXX

**수강료:** 50,000원

**신청 방법:** 아래의 이메일로 신청서를 제출해 주시기 바랍니다. 선착순 마감입니다. XXXXXXX@example.com

**주최자 연락처:** 전화번호 010-XXXXX-XXXXX, 카카오톡 ID: XXXXXXXXXXX, 트위터: @XXXXXXXXXXXXX, 텔레그램: @XXXXXXXXXXXXX

**강의 상세 내용:** 본 강의는 데이터 분석의 기본 개념과 Python 프로그래밍 언어를 활용하여 데이터 분석을 수행하는 방법을 배우는 프로그램입니다. 강의는 데이터 전처리, 시각화 및 분석 기법을 포함하며, 실습을 통해 이론을 체득할 수 있도록 구성되어 있습니다. 초보자도 쉽게 따라할 수 있도록 단계별로 설명하며, 실제 사례를 통해 학습의 이해도를 높입니다. 

데이터 분석에 관심 있는 분들, 취업 준비생 및 직장인 분들의 많은 참여를 바랍니다.


In [53]:
import re

def split_korean_particles(tokens):
    new_tokens = []
    for tok in tokens:
        m = re.match(r'^([A-Za-z0-9@._-]+)(으로|로)$', tok)
        if m:
            stem, particle = m.groups()
            new_tokens.append(stem)
            new_tokens.append(particle)
        else:
            new_tokens.append(tok)
    return new_tokens

import re

def custom_tokenize(text):
    pattern = (
        r'\*\*[^*]+\*\*'       # 1) '**...**' 구간
        r'|\*[^*]+\*'          # 2) '*...*' 구간
        r'|[A-Za-z0-9@._-]+'   # 3) 영숫자/@._-
        r'|[가-힣]+'           # 4) 한글 연속
        r'|\S'                 # 5) 그 외 공백 아닌 문자 1글자
    )
    tokens = re.findall(pattern, text)
    return tokens


def custom_tokenize_with_placeholder(text):
    """
    - '*[PHONE_NUMBER]*' 등 별표+플레이스홀더를 먼저 한 토큰으로 매칭
    - 'xxx -> [PHONE_NUMBER]' 등도 한 토큰
    - 그 외 '**text**', '*text*' 등을 매칭
    - 나머지는 [A-Za-z0-9@._-]+, [가-힣]+, \S 
    """
    pattern = (
        r'\*\[[A-Z_]+\]\*'  # 1) 예: *[PHONE_NUMBER]*, *[EMAIL]*, *[ADDR]* 등
        r'|[A-Za-z0-9@._-]+\s*->\s*\[(?:ADDR|PHONE_NUMBER|EMAIL|ID)\]'
        r'|\*\*[^*]+\*\*'   # 2) '**some**'
        r'|\*[^*]+\*'       # 3) '*some*'
        r'|[A-Za-z0-9@._-]+'
        r'|[가-힣]+'
        r'|\S'
    )
    tokens = re.findall(pattern, text)
    return tokens

def label_original_tokens(tokens):
    """
    - 전화번호: 02-12345, 010-xxxxx-xxxx, etc. (간단 예시)
    - 이메일: something@something.com
    - ID: 'sudo' 문자열을 포함하면 B-ID
    - 주소: '*...*' 형태의 토큰에, 내부 공백이 2번 이상 있으면 B-ADDR
    """
    labels = []
    
    # 예: 02-12345, 02-12345-6789, 010-xxxx-xxxx ...
    phone_pattern = re.compile(r'^\d{2,3}-\d{4,5}(-\d{4,5})?$')
    email_pattern = re.compile(r'^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]+$')
    addr_pattern = re.compile(r'^\*[^*]+\*$')  # "단일 별표" 패턴

    for t in tokens:
        clean_t = t.strip("*")  # 별표 제거 후 확인용
        # (A) 전화번호
        if phone_pattern.match(clean_t):
            labels.append("B-PHONE")
        # (B) 이메일
        elif email_pattern.match(clean_t):
            labels.append("B-EMAIL")
        # (C) ID (clean_t 안에 'sudo' 문자가 있으면)
        elif "sudo" in clean_t:
            labels.append("B-ID")
        # (D) 주소 (단일 별표 + 내부 공백 2개 이상)
        elif addr_pattern.match(t):
            # t는 '*서울시 마포구 상암동*' 처럼 시작/끝 각각 별표 1개
            space_count = clean_t.count(" ")
            if space_count >= 2:
                labels.append("B-ADDR")
            else:
                labels.append("O")
        else:
            labels.append("O")
    return labels


def label_anonymized_tokens(tokens):
    """
    - [PHONE_NUMBER] -> B-PHONE
    - [EMAIL] -> B-EMAIL
    - [ID] -> B-ID
    - [ADDR] -> B-ADDR
    """
    placeholder_map = {
        "ADDR": "B-ADDR",
        "PHONE_NUMBER": "B-PHONE",
        "EMAIL": "B-EMAIL",
        "ID": "B-ID",
    }
    
    labels = []
    for t in tokens:
        clean_t = t.strip("*")
        
        # 부분 문자열 "[ADDR]" → B-ADDR, etc.
        if "[ADDR]" in clean_t:
            labels.append("B-ADDR")
        elif "[PHONE_NUMBER]" in clean_t:
            labels.append("B-PHONE")
        elif "[EMAIL]" in clean_t:
            labels.append("B-EMAIL")
        elif "[ID]" in clean_t:
            labels.append("B-ID")
        else:
            if clean_t in placeholder_map:
                labels.append(placeholder_map[clean_t])
            else:
                labels.append("O")
    return labels


def calculate_metrics(orig_labels, anon_labels):
    tp = 0
    fp = 0
    fn = 0
    
    length = min(len(orig_labels), len(anon_labels))
    for i in range(length):
        o_label = orig_labels[i]
        a_label = anon_labels[i]
        
        if o_label.startswith("B-"):
            if a_label == o_label:
                tp += 1
            else:
                fn += 1
        else:
            if a_label.startswith("B-"):
                fp += 1
    
    precision = tp / (tp + fp) if (tp+fp) else 0
    recall = tp / (tp + fn) if (tp+fn) else 0
    f1 = 2*precision*recall / (precision+recall) if (precision+recall) else 0
    return precision, recall, f1

# ---------------------------
# 실제 실행
original_text = """🌟 **제1회 청소년 환경 보호 캠페인** 🌟

✅ **일시**: 2023년 11월 20일 (월) 오전 10시부터 오후 4시까지  
✅ **장소**: *서울시 종로구 청계천로 123, 청계광장*  

👥 **참가 방법**: 사전 신청 필수! 아래의 이메일로 신청해 주세요.  
*en_sudo_user@example.com*  

📞 **주최자 연락처**: 궁금한 점이 있으시면 언제든지 연락 주세요. 전화번호는 *010-87590-12345* 입니다.  

🌍 **이벤트 상세 내용**:  
이번 캠페인은 청소년들이 환경 보호의 중요성을 이해하고 실천할 수 있도록 돕기 위해 마련되었습니다. 다양한 프로그램이 준비되어 있으며, 특히 다음과 같은 활동이 포함됩니다:  
- 환경 전문가의 강연  
- 쓰레기 줍기 및 생태 보호 활동  
- 재활용 DIY 체험 부스  
- 퀴즈 및 게임을 통한 환경 교육  

👫 이 캠페인은 청소년들뿐만 아니라 가족 모두가 함께 참여할 수 있는 프로그램으로, 환경 문제에 대한 이해와 실천을 촉진하는 기회가 될 것입니다.  

💬 더 많은 정보는 카카오톡 아이디 *eco_camp_sudo* 또는 트위터 *@greenfuture_sudo* 를 통해 확인하실 수 있습니다.  

여러분의 많은 관심과 참여를 부탁드립니다! 함께 지구를 지켜요! 🌱"""

anonymized_text = """🌟 **제1회 청소년 환경 보호 캠페인** 🌟

✅ **일시**: 2023년 11월 20일 (월) 오전 10시부터 오후 4시까지  
✅ **장소**: *[ADDR]*  

👥 **참가 방법**: 사전 신청 필수! 아래의 이메일로 신청해 주세요.  
*[EMAIL]*  

📞 **주최자 연락처**: 궁금한 점이 있으시면 언제든지 연락 주세요. 전화번호는 *[PHONE_NUMBER]* 입니다.  

🌍 **이벤트 상세 내용**:  
이번 캠페인은 청소년들이 환경 보호의 중요성을 이해하고 실천할 수 있도록 돕기 위해 마련되었습니다. 다양한 프로그램이 준비되어 있으며, 특히 다음과 같은 활동이 포함됩니다:  
- 환경 전문가의 강연  
- 쓰레기 줍기 및 생태 보호 활동  
- 재활용 DIY 체험 부스  
- 퀴즈 및 게임을 통한 환경 교육  

👫 이 캠페인은 청소년들뿐만 아니라 가족 모두가 함께 참여할 수 있는 프로그램으로, 환경 문제에 대한 이해와 실천을 촉진하는 기회가 될 것입니다.  

💬 더 많은 정보는 카카오톡 아이디 *[ID]* 또는 트위터 *[ID]* 를 통해 확인하실 수 있습니다.  

여러분의 많은 관심과 참여를 부탁드립니다! 함께 지구를 지켜요! 🌱"""

orig_tokens = custom_tokenize(original_text)
anon_tokens = custom_tokenize_with_placeholder(anonymized_text)

orig_labels = label_original_tokens(orig_tokens)
anon_labels = label_anonymized_tokens(anon_tokens)

p, r, f1 = calculate_metrics(orig_labels, anon_labels)
print("Precision:", p, "Recall:", r, "F1:", f1)


Precision: 1.0 Recall: 1.0 F1: 1.0
